In [3]:
import numpy as np
import numpy.linalg as la

## 优化方法

### 1. 梯度下降法

一阶优化算法（仅使用梯度信息）

In [2]:
# 实例：线性最小二乘
# 问题：给定一个矩阵A和一个向量b，找到一个向量x，使得f(x) = 0.5 * ||Ax - b||_2^2最小
# 可以计算梯度为：grad(f(x)) = A^T(Ax - b) = A^TAx - A^Tb

x0 = np.array([1.0, 1.0, 1.0])
A = np.array([[1.0, -2.0, 1.0], [0.0, 2.0, -8.0], [-4.0, 5.0, 9.0]])
b = np.array([0.0, 8.0, -9.0])
epsilon = 0.001  # learning rate
delta = 1e-3  # convergence criterion
# 给定A和b，真正的解x是[29.0, 16.0, 3.0]

In [4]:
def matual_chain(*args):
    """矩阵链乘积"""
    if len(args) == 0:
        return np.nan
    result = args[0]
    for i in range(1, len(args)):
        result = np.dot(result, args[i])
        # reuslt = result@args[i]
    return result

def gradient_descent(x, A, b, epsilon, delta):
    while la.norm(matual_chain(A.T, A, x) - matual_chain(A.T, b)) > delta:  # 梯度的范数 > delta（函数在当前点的变化率还较大）
        x -= epsilon * (matual_chain(A.T, A, x) - matual_chain(A.T, b))  # x' = x - epsilon * grad (梯度下降迭代)
    return x

In [5]:
gradient_descent(x0, A, b, epsilon, delta)  # 很接近真正解[29.0, 16.0, 3.0]

array([27.82277014, 15.34731055,  2.83848939])

### 2. 牛顿法

二阶优化算法（使用梯度 和 Hessian矩阵（二阶导信息））

- 牛顿法迭代地更新近似函数和跳到近似函数的最小点，可以比梯度下降法更快地收敛到最小值
- 这在接近全局最小值时特别有用，但是在鞍点附近可能时有害的

牛顿法基于一个二阶泰勒展开来近似x^0附近的函数，然后求解这个近似函数的最小值，得到下一个迭代点$x^1$，然后重复这个过程直到收敛:

$f(x) ≈ f(x^0) + ∇f(x^0)(x - x^0) + 1/2(x - x^0)^T H(x^0)(x - x^0)$

接着通过计算，我们可以得到这个函数的临界点：

$x^* = x^0 - H(x^0)^-1∇f(x^0)$

针对上述实例（线性最小二乘），计算得到$H = A^TA$，进一步计算得到最优解：

$x^* = x^0 - (A^TA)^{-1}(A^TAx^0 - A^Tb) = (A^TA)^{-1}A^Tb$

In [6]:
def newton(x, A, b, delta):
    x = matual_chain(la.inv(matual_chain(A.T, A)), matual_chain(A.T, b))  # x = inv(H) * grad (牛顿法迭代), inv()是求逆
    return x

In [7]:
newton(x0, A, b, delta)  # 等于真正解[29.0, 16.0, 3.0]

array([29., 16.,  3.])

### 3. 约束优化

![constrained optimization](images/constrained_optim.png)

In [8]:
def constrain_optim(x, A, b, delta=5e-2):
    k = len(x)
    lamb = 0
    while np.abs(np.dot(x.T, x)-1) > delta:  #  delta设为5e-2，最优设为0
        x = matual_chain(la.inv(matual_chain(A.T, A) + 2*lamb*np.identity(k)), matual_chain(A.T, b))
        lamb += np.dot(x.T, x)-1
    return x

In [9]:
constrain_optim(x0, A, b, delta=5e-2)

array([ 0.23637902,  0.05135858, -0.94463626])